# Normalising Covariance

This notepad looks at online normalisation of the covariance matrix to generate a correlation matrix.

The normalisation of the covariance matrix involves dividing by the standard deviations. But the variances (SD^2) are the normals of the covariance matrix. Hence, to normalise we can perform:

CORR = COV // (D_dash.D_dash^T)

where D_dash = vector of the standard deviations - or np.sqrt(np.diag(COV)).

This notebook will check that this works and matches the correlation matrices. Numpy's correlation function is [numpy.corrcoef()](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html) and it's covariance function is [numpy.cov()](https://numpy.org/doc/stable/reference/generated/numpy.cov.html#numpy.cov).

In [1]:
import numpy as np

In [12]:
# Generate some random data - 100 samples of 8 features.
# Make the samples centred around 0 so we don't have to remove the mean
rand_samples = np.random.randint(low=-127, high=128, size=(8, 100))

In [13]:
np_cov = np.cov(rand_samples).astype(np.int16)

In [14]:
np_cov

array([[ 5319,  -526,   685,  -397,   858,  1297,   160,   191],
       [ -526,  5024,   902,  -469,   537,   -57,  -319,   -90],
       [  685,   902,  5292,   263,   225,  -409,  -337,   144],
       [ -397,  -469,   263,  6001,  -347,  -858,  1006, -1258],
       [  858,   537,   225,  -347,  5344,    -9,    71,  1022],
       [ 1297,   -57,  -409,  -858,    -9,  5236,  -393,    90],
       [  160,  -319,  -337,  1006,    71,  -393,  5653, -1221],
       [  191,   -90,   144, -1258,  1022,    90, -1221,  5126]],
      dtype=int16)

In [15]:
np_corr = np.corrcoef(rand_samples)

In [16]:
np.around(np_corr, 3)

array([[ 1.   , -0.102,  0.129, -0.07 ,  0.161,  0.246,  0.029,  0.037],
       [-0.102,  1.   ,  0.175, -0.086,  0.104, -0.011, -0.06 , -0.018],
       [ 0.129,  0.175,  1.   ,  0.047,  0.042, -0.078, -0.062,  0.028],
       [-0.07 , -0.086,  0.047,  1.   , -0.061, -0.153,  0.173, -0.227],
       [ 0.161,  0.104,  0.042, -0.061,  1.   , -0.002,  0.013,  0.195],
       [ 0.246, -0.011, -0.078, -0.153, -0.002,  1.   , -0.072,  0.018],
       [ 0.029, -0.06 , -0.062,  0.173,  0.013, -0.072,  1.   , -0.227],
       [ 0.037, -0.018,  0.028, -0.227,  0.195,  0.018, -0.227,  1.   ]])

## Test Normalising Covariance

Let's start by checking that cov / D_dash.D_dash^T = Corr

In [37]:
d_dash = np.sqrt(np.diag(np_cov)).reshape(8, 1); print(f"D_dash from cov:\n {d_dash.T}")

D_dash from cov:
 [[72.93147 70.88018 72.74613 77.46612 73.10267 72.36021 75.18643 71.59609]]


In [39]:
np.around(np_cov / np.dot(d_dash, d_dash.T), 3)

array([[ 1.   , -0.102,  0.129, -0.07 ,  0.161,  0.246,  0.029,  0.037],
       [-0.102,  1.   ,  0.175, -0.085,  0.104, -0.011, -0.06 , -0.018],
       [ 0.129,  0.175,  1.   ,  0.047,  0.042, -0.078, -0.062,  0.028],
       [-0.07 , -0.085,  0.047,  1.   , -0.061, -0.153,  0.173, -0.227],
       [ 0.161,  0.104,  0.042, -0.061,  1.   , -0.002,  0.013,  0.195],
       [ 0.246, -0.011, -0.078, -0.153, -0.002,  1.   , -0.072,  0.017],
       [ 0.029, -0.06 , -0.062,  0.173,  0.013, -0.072,  1.   , -0.227],
       [ 0.037, -0.018,  0.028, -0.227,  0.195,  0.017, -0.227,  1.   ]],
      dtype=float32)

Yes - this works, we get the correlation matrix.

---
## Online Correlation

Now we try an online version.

First a dry run of the steps.

In [45]:
# Matrix to hold running covariance
running_cov = np.zeros(shape=(8, 8))

In [46]:
i = 0
# Get ith sample
X = rand_samples[:, i].reshape(8, 1)
# Determine X.X^T
sum_sample = np.dot(X, X.T)
print(f"X = \n {X.T}\n")
print(f"X.X^T = \n {sum_sample}")

X = 
 [[  30   82  -76 -104   -3   40   47   40]]

X.X^T = 
 [[  900  2460 -2280 -3120   -90  1200  1410  1200]
 [ 2460  6724 -6232 -8528  -246  3280  3854  3280]
 [-2280 -6232  5776  7904   228 -3040 -3572 -3040]
 [-3120 -8528  7904 10816   312 -4160 -4888 -4160]
 [  -90  -246   228   312     9  -120  -141  -120]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]
 [ 1410  3854 -3572 -4888  -141  1880  2209  1880]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]]


In [47]:
d = np.diag(sum_sample).reshape(8, 1); print(f"Diagonals - variance of elements - \n {d.T}")  

Diagonals - variance of elements - 
 [[  900  6724  5776 10816     9  1600  2209  1600]]


In [48]:
d_dash = np.sqrt(d).reshape(8, 1); print(f"Standard deviation - sqrt(var) - \n {d_dash.T}")  

Standard deviation - sqrt(var) - 
 [[ 30.  82.  76. 104.   3.  40.  47.  40.]]


In [51]:
# Set printing precision to 3 decimal places
np.set_printoptions(precision=3, suppress=True)
print(f"Initial covariance:\n {running_cov}")
running_cov += sum_sample
d = np.diag(running_cov); print(f"Diagonals - variance of elements - \n {d}")
d_dash = np.sqrt(d).reshape(8, 1); print(f"Standard deviation - sqrt(var) - \n {d_dash}")
normalisation_mat = np.dot(d_dash, d_dash.T); print(f"Normalisation matrix: \n {normalisation_mat}")
running_cov = running_cov / np.dot(d_dash, d_dash.T)
print(f"Subsequent scaled covariance:\n {running_cov}")

Initial covariance:
 [[ 0.03   0.083 -0.077 -0.105 -0.003  0.04   0.048  0.04 ]
 [ 0.083  0.227 -0.21  -0.288 -0.008  0.111  0.13   0.111]
 [-0.077 -0.21   0.195  0.267  0.008 -0.103 -0.121 -0.103]
 [-0.105 -0.288  0.267  0.365  0.011 -0.14  -0.165 -0.14 ]
 [-0.003 -0.008  0.008  0.011  0.    -0.004 -0.005 -0.004]
 [ 0.04   0.111 -0.103 -0.14  -0.004  0.054  0.063  0.054]
 [ 0.048  0.13  -0.121 -0.165 -0.005  0.063  0.075  0.063]
 [ 0.04   0.111 -0.103 -0.14  -0.004  0.054  0.063  0.054]]
Diagonals - variance of elements - 
 [  900.03   6724.227  5776.195 10816.365     9.     1600.054  2209.075
  1600.054]
Standard deviation - sqrt(var) - 
 [[ 30.001]
 [ 82.001]
 [ 76.001]
 [104.002]
 [  3.   ]
 [ 40.001]
 [ 47.001]
 [ 40.001]]
Normalisation matrix: 
 [[  900.03   2460.083  2280.077  3120.105    90.003  1200.04   1410.048
   1200.04 ]
 [ 2460.083  6724.227  6232.21   8528.288   246.008  3280.111  3854.13
   3280.111]
 [ 2280.077  6232.21   5776.195  7904.267   228.008  3040.103  3572.1

We need to be careful. If we just scale then take as the next covariance, we'll be adding very high ranges (e.g. 10k etc to 1, -1 values). So we need to scale then take the average?

***Scaling just keeps the sign. So maybe that is all we are interested in...***



In [52]:
# Matrix to hold running covariance
running_cov = np.zeros(shape=(8, 8))

for i in range(100):
    # Get ith sample
    X = rand_samples[:, i].reshape(8, 1)
    # Determine X.X^T
    sum_sample = np.dot(X, X.T)
    print(f"X = \n {X.T}\n")
    print(f"X.X^T = \n {sum_sample}")
    print(f"Initial covariance:\n {running_cov}")
    running_cov += sum_sample
    d = np.diag(running_cov); print(f"Diagonals - variance of elements - \n {d}")
    d_dash = np.sqrt(d).reshape(8, 1); print(f"Standard deviation - sqrt(var) - \n {d_dash}")
    normalisation_mat = np.dot(d_dash, d_dash.T); print(f"Normalisation matrix: \n {normalisation_mat}")
    running_cov = running_cov / np.dot(d_dash, d_dash.T)
    print(f"Subsequent scaled covariance:\n {running_cov}", end="\n---\n")

X = 
 [[  30   82  -76 -104   -3   40   47   40]]

X.X^T = 
 [[  900  2460 -2280 -3120   -90  1200  1410  1200]
 [ 2460  6724 -6232 -8528  -246  3280  3854  3280]
 [-2280 -6232  5776  7904   228 -3040 -3572 -3040]
 [-3120 -8528  7904 10816   312 -4160 -4888 -4160]
 [  -90  -246   228   312     9  -120  -141  -120]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]
 [ 1410  3854 -3572 -4888  -141  1880  2209  1880]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]]
Initial covariance:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
Diagonals - variance of elements - 
 [  900.  6724.  5776. 10816.     9.  1600.  2209.  1600.]
Standard deviation - sqrt(var) - 
 [[ 30.]
 [ 82.]
 [ 76.]
 [104.]
 [  3.]
 [ 40.]
 [ 47.]
 [ 40.]]
Normalisation matrix: 
 [[  900.  2460.  2280.  3120.    90.  1200.  1410.  1200.]
 [ 2460.  6724

 [-11856  -3762  12198  12882   5016  -5472  12198  12996]]
Initial covariance:
 [[ 1.    -1.    -0.997 -1.    -1.    -0.011  1.     0.999]
 [-1.     1.     0.998  1.     1.    -0.014 -0.999 -1.   ]
 [-0.997  0.998  1.     0.997  0.997 -0.071 -0.994 -0.999]
 [-1.     1.     0.997  1.     1.     0.008 -1.    -0.999]
 [-1.     1.     0.997  1.     1.     0.009 -1.    -0.999]
 [-0.011 -0.014 -0.071  0.008  0.009  1.    -0.034  0.028]
 [ 1.    -0.999 -0.994 -1.    -1.    -0.034  1.     0.998]
 [ 0.999 -1.    -0.999 -0.999 -0.999  0.028  0.998  1.   ]]
Diagonals - variance of elements - 
 [10817.  1090. 11450. 12770.  1937.  2305. 11450. 12997.]
Standard deviation - sqrt(var) - 
 [[104.005]
 [ 33.015]
 [107.005]
 [113.004]
 [ 44.011]
 [ 48.01 ]
 [107.005]
 [114.004]]
Normalisation matrix: 
 [[10817.     3433.734 11129.    11753.003  4577.393  4993.314 11129.
  11857.004]
 [ 3433.734  1090.     3532.775  3730.858  1453.042  1585.071  3532.775
   3763.872]
 [11129.     3532.775 11450.    1209

So the normalisation matrix is always positive - so be dividing by the absolute of the value, all you get left with is the sign.

Let's check this below:

In [53]:
# Matrix to hold running covariance
running_cov = np.zeros(shape=(8, 8))

for i in range(100):
    # Get ith sample
    X = rand_samples[:, i].reshape(8, 1)
    # Determine X.X^T
    sum_sample = np.dot(X, X.T)
    print(f"X = \n {X.T}\n")
    print(f"X.X^T = \n {sum_sample}")
    d = np.diag(sum_sample); # print(f"Diagonals - variance of elements - \n {d}")
    d_dash = np.sqrt(d).reshape(8, 1); # print(f"Standard deviation - sqrt(var) - \n {d_dash}")
    normalisation_mat = np.dot(d_dash, d_dash.T); # print(f"Normalisation matrix: \n {normalisation_mat}")
    normalised = sum_sample / np.dot(d_dash, d_dash.T)
    print(f"Normalised Matrix:\n {normalised}")
    signs = np.sign(sum_sample)
    print(f"Signs:\n {signs}")
    print("", end="\n---\n")

X = 
 [[  30   82  -76 -104   -3   40   47   40]]

X.X^T = 
 [[  900  2460 -2280 -3120   -90  1200  1410  1200]
 [ 2460  6724 -6232 -8528  -246  3280  3854  3280]
 [-2280 -6232  5776  7904   228 -3040 -3572 -3040]
 [-3120 -8528  7904 10816   312 -4160 -4888 -4160]
 [  -90  -246   228   312     9  -120  -141  -120]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]
 [ 1410  3854 -3572 -4888  -141  1880  2209  1880]
 [ 1200  3280 -3040 -4160  -120  1600  1880  1600]]
Normalised Matrix:
 [[ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [-1. -1.  1.  1.  1. -1. -1. -1.]
 [-1. -1.  1.  1.  1. -1. -1. -1.]
 [-1. -1.  1.  1.  1. -1. -1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]]
Signs:
 [[ 1  1 -1 -1 -1  1  1  1]
 [ 1  1 -1 -1 -1  1  1  1]
 [-1 -1  1  1  1 -1 -1 -1]
 [-1 -1  1  1  1 -1 -1 -1]
 [-1 -1  1  1  1 -1 -1 -1]
 [ 1  1 -1 -1 -1  1  1  1]
 [ 1  1 -1 -1 -1  1  1  1]
 [ 1  1 -1 -1 -1  1  1  1]]

---

Normalised Matrix:
 [[ 1. -1. -1.  1.  1. -1.  1. -1.]
 [-1.  1.  1. -1. -1.  1. -1.  1.]
 [-1.  1.  1. -1. -1.  1. -1.  1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.]
 [-1.  1.  1. -1. -1.  1. -1.  1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.]
 [-1.  1.  1. -1. -1.  1. -1.  1.]]
Signs:
 [[ 1 -1 -1  1  1 -1  1 -1]
 [-1  1  1 -1 -1  1 -1  1]
 [-1  1  1 -1 -1  1 -1  1]
 [ 1 -1 -1  1  1 -1  1 -1]
 [ 1 -1 -1  1  1 -1  1 -1]
 [-1  1  1 -1 -1  1 -1  1]
 [ 1 -1 -1  1  1 -1  1 -1]
 [-1  1  1 -1 -1  1 -1  1]]

---


/home/ben/anaconda3/envs/var_processor/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


## Revised Method

In [54]:
# Matrix to hold running covariance
running_cov = np.zeros(shape=(8, 8))

for i in range(100):
    # Get ith sample
    X = rand_samples[:, i].reshape(8, 1)
    # Determine X.X^T
    sum_sample = np.dot(X, X.T)
    signs = np.sign(sum_sample)
    running_cov += signs

print(f"Summed Covariance:\n{running_cov}")
print(f"Scaled Covariance:\n{running_cov/100}")

Summed Covariance:
[[100.  -7.  12.  -6.  24.  17.  -1.   6.]
 [ -7.  99.   5.  -7.  25.  12.   0.  -5.]
 [ 12.   5. 100.   0.  -4.  -5.  -1.  -6.]
 [ -6.  -7.   0.  98.  -2.  -9.  13. -14.]
 [ 24.  25.  -4.  -2. 100.   3.  -1.  10.]
 [ 17.  12.  -5.  -9.   3.  99.   4.   3.]
 [ -1.   0.  -1.  13.  -1.   4.  99. -23.]
 [  6.  -5.  -6. -14.  10.   3. -23. 100.]]
Scaled Covariance:
[[ 1.   -0.07  0.12 -0.06  0.24  0.17 -0.01  0.06]
 [-0.07  0.99  0.05 -0.07  0.25  0.12  0.   -0.05]
 [ 0.12  0.05  1.    0.   -0.04 -0.05 -0.01 -0.06]
 [-0.06 -0.07  0.    0.98 -0.02 -0.09  0.13 -0.14]
 [ 0.24  0.25 -0.04 -0.02  1.    0.03 -0.01  0.1 ]
 [ 0.17  0.12 -0.05 -0.09  0.03  0.99  0.04  0.03]
 [-0.01  0.   -0.01  0.13 -0.01  0.04  0.99 -0.23]
 [ 0.06 -0.05 -0.06 -0.14  0.1   0.03 -0.23  1.  ]]


In [55]:
np_corr

array([[ 1.   , -0.102,  0.129, -0.07 ,  0.161,  0.246,  0.029,  0.037],
       [-0.102,  1.   ,  0.175, -0.086,  0.104, -0.011, -0.06 , -0.018],
       [ 0.129,  0.175,  1.   ,  0.047,  0.042, -0.078, -0.062,  0.028],
       [-0.07 , -0.086,  0.047,  1.   , -0.061, -0.153,  0.173, -0.227],
       [ 0.161,  0.104,  0.042, -0.061,  1.   , -0.002,  0.013,  0.195],
       [ 0.246, -0.011, -0.078, -0.153, -0.002,  1.   , -0.072,  0.018],
       [ 0.029, -0.06 , -0.062,  0.173,  0.013, -0.072,  1.   , -0.227],
       [ 0.037, -0.018,  0.028, -0.227,  0.195,  0.018, -0.227,  1.   ]])

In [57]:
error = np_corr*100 - running_cov; print(error)

[[  0.     -3.18    0.913  -1.039  -7.896   7.592   3.918  -2.337]
 [ -3.18    1.     12.507  -1.554 -14.621 -13.129  -6.003   3.223]
 [  0.913  12.507   0.      4.68    8.241  -2.774  -5.169   8.769]
 [ -1.039  -1.554   4.68    2.     -4.13   -6.306   4.274  -8.693]
 [ -7.896 -14.621   8.241  -4.13    0.     -3.172   2.308   9.537]
 [  7.592 -13.129  -2.774  -6.306  -3.172   1.    -11.225  -1.248]
 [  3.918  -6.003  -5.169   4.274   2.308 -11.225   1.      0.313]
 [ -2.337   3.223   8.769  -8.693   9.537  -1.248   0.313   0.   ]]


In [59]:
error/(np_corr*100)

array([[ 0.   ,  0.312,  0.071,  0.148, -0.49 ,  0.309,  1.343, -0.638],
       [ 0.312,  0.01 ,  0.714,  0.182, -1.409, 11.631,  1.   , -1.814],
       [ 0.071,  0.714,  0.   ,  1.   ,  1.943,  0.357,  0.838,  3.167],
       [ 0.148,  0.182,  1.   ,  0.02 ,  0.674,  0.412,  0.247,  0.383],
       [-0.49 , -1.409,  1.943,  0.674,  0.   , 18.411,  1.765,  0.488],
       [ 0.309, 11.631,  0.357,  0.412, 18.411,  0.01 ,  1.554, -0.712],
       [ 1.343,  1.   ,  0.838,  0.247,  1.765,  1.554,  0.01 , -0.014],
       [-0.638, -1.814,  3.167,  0.383,  0.488, -0.712, -0.014,  0.   ]])

In [60]:
ones = np.ones(shape=(8, 1))
print(np.dot(np_corr, ones), np.dot((running_cov/100), ones), sep="\n----\n")

[[1.43 ]
 [1.002]
 [1.282]
 [0.622]
 [1.453]
 [0.947]
 [0.794]
 [0.806]]
----
[[1.45]
 [1.22]
 [1.01]
 [0.73]
 [1.55]
 [1.24]
 [0.9 ]
 [0.71]]


In [61]:
# What are the eigenvectors of both estimates
# The sign correlation
w1, v2 = np.linalg.eig(running_cov/100)
# Check eigenvectors 
print(abs(v[:, np.argmax(w)]))
# Check eigenvalues 
print(w[np.argmax(w)])
# What are the eigenvectors of both estimates
w2, v2 = np.linalg.eig(np_corr)
# Check eigenvectors 
print(abs(v[:, np.argmax(w)]))
# Check eigenvalues 
print(w[np.argmax(w)])

[0.438 0.293 0.021 0.362 0.503 0.326 0.279 0.393]
1.470801261535347
[0.309 0.139 0.123 0.494 0.34  0.313 0.397 0.505]
1.5667428972210033
